In [ ]:
import os
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAAIi0jQEAAAAAk2nnTNI41oHHdehJLOXXv36J6%2F0%3DnnMdA329Tb9YGoAo6vgGLjvEVG0m8WwCs3ZHQNqGHLC2fUdz3v"

In [ ]:
import requests 
import pandas as pd 
import time

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
headers = create_headers(os.environ['TOKEN'])

In [ ]:
def create_url(query, start_time, end_time, max_results, expansions, tweet_fields, user_fields, place_fields, endpoint):
    
    search_url = endpoint #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    #also can request different fields, e.g ids of users ... 
    query_params = {'query': query,
                    'end_time': end_time,
                    'start_time': start_time,
                    'max_results': max_results,
                    'expansions': expansions,
                    'tweet.fields': tweet_fields,
                    'user.fields': user_fields,
                    'place.fields': place_fields}

    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    #only change the default value of next_token if it is a real value returned in the response
    if next_token is not None and next_token != '':
      params['next_token'] = next_token
    #create a "GET" request to the specified url, add headers and parameters
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        #if something goes wrong, we need to know
        raise Exception(response.status_code, response.text)
    #otherwise, we want the payload of our response, which contains our tweet(s)
    return response.json()

In [ ]:
def get_data(query, start_time, end_time, max_results, expansions, tweet_fields, user_fields, place_fields, endpoint, next_token=""):
  
  results = []

  loop=0
  while next_token is not None and loop<10000:
    try:    
      url = create_url(query, start_time, end_time, max_results, expansions, tweet_fields, user_fields, place_fields, endpoint)
      json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
      #if we have results, they will be in the field 'data' of our response
      print(json_response.keys())
      if "data" in json_response:
        results.extend(json_response['data'])
        
        print(str(len(json_response["data"])) + " Tweets downloaded in this batch.")
      #the next_token is added to the field 'meta' of our response
      if "meta" in json_response:
        if "next_token" in json_response["meta"].keys():
          next_token = json_response["meta"]["next_token"]          
        else:
          next_token = None
      else:
        next_token = None

      
      #to control the rate limit we need to slow down our download
      time.sleep(2)
      loop+=1
      print(loop)

    except Exception as e:
      print("Error occured", e)
      print("Next token value", next_token)
      error_log = {"Error":e, "Next token":next_token, "Day":start_time, 
                   "Downloaded":len(results)}
      pd.DataFrame.from_dict(error_log, orient="index").to_csv(query+"_"+start_time+"_"+next_token+".csv")
      return(results, next_token)

  print("Done")
  
  return results

In [ ]:
tweets = get_data("#chatGPT", start_time = "2023-04-21T00:00:00Z", end_time = "2023-04-27T00:00:00Z", 
         max_results=100, expansions='author_id,in_reply_to_user_id,geo.place_id', 
         tweet_fields='id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,context_annotations,entities',
         user_fields='id,name,username,created_at,description,public_metrics,verified',
         place_fields='full_name,id,country,country_code,geo,name,place_type',
         endpoint="https://api.twitter.com/2/tweets/search/recent")

dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
1
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
2
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
3
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
4
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
5
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
6
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
7
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
8
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
9
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
10
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
11
dict_keys(['data', 'includes', 'meta'])
100 Tweets downloaded in this batch.
12
dict_keys(['data', 'includes', 'meta'])
100 Tweet

In [ ]:
tweets_df = pd.DataFrame(tweets)

In [ ]:
display(tweets_df)

,context_annotations,reply_settings,text,public_metrics,edit_history_tweet_ids,created_at,id,conversation_id,entities,lang,author_id,referenced_tweets,in_reply_to_user_id,geo,withheld
0,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",everyone,さっきの偽物販売同様結局正しいデータだわ 誤った情報がばら撒かられる フェイク情報で販売促進...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",[1651375357298364416],2023-04-26T23:59:00.000Z,1651375357298364416,1651375357298364416,"{'hashtags': [{'start': 54, 'end': 63, 'tag': ...",ja,67909514,NaN,NaN,NaN,NaN
1,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,RT @scispace_: Este asistente basado en ChatGP...,"{'retweet_count': 424, 'reply_count': 0, 'like...",[1651375352705429504],2023-04-26T23:58:59.000Z,1651375352705429504,1651375352705429504,"{'annotations': [{'start': 40, 'end': 46, 'pro...",es,1560350125242961921,"[{'type': 'retweeted', 'id': '1639307586724937...",NaN,NaN,NaN
2,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,RT @AIFloki_io: AI Floki 🤖🐶 follows in the foo...,"{'retweet_count': 1537, 'reply_count': 0, 'lik...",[1651375265925197824],2023-04-26T23:58:39.000Z,1651375265925197824,1651375265925197824,"{'annotations': [{'start': 19, 'end': 23, 'pro...",en,1643786374519070721,"[{'type': 'retweeted', 'id': '1625831302355357...",NaN,NaN,NaN
3,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,RT @inworld_ai: How would #AI characters enhan...,"{'retweet_count': 5, 'reply_count': 0, 'like_c...",[1651375264319021056],2023-04-26T23:58:38.000Z,1651375264319021056,1651375264319021056,"{'annotations': [{'start': 27, 'end': 28, 'pro...",en,56382584,"[{'type': 'retweeted', 'id': '1651300186973405...",NaN,NaN,NaN
4,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,🤩🤩🤩🤩 #仙境傳說 #英雄聯盟 #tendate #館長 #chatgpt \nSour...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",[1651375219615866880],2023-04-26T23:58:28.000Z,1651375219615866880,1651375219615866880,"{'urls': [{'start': 49, 'end': 72, 'url': 'htt...",en,1616657002817527810,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95727,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,RT @KidsCampFes: 昨晩開催された倒木事故に関するCAMP TEBA2号店のス...,"{'retweet_count': 21, 'reply_count': 0, 'like_...",[1649201333009022976],2023-04-21T00:00:13.000Z,1649201333009022976,1649201333009022976,"{'annotations': [{'start': 51, 'end': 55, 'pro...",ja,1522211717614227457,"[{'type': 'retweeted', 'id': '1649169354834206...",NaN,NaN,NaN
95728,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,¡CAPACITACIÓN GRATUITA! Lo que no Sabías de C...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",[1649201326192992257],2023-04-21T00:00:11.000Z,1649201326192992257,1649201326192992257,"{'annotations': [{'start': 45, 'end': 51, 'pro...",es,185954038,NaN,NaN,NaN,NaN
95729,"[{'domain': {'id': '46', 'name': 'Business Tax...",everyone,【チャットGPT物語】\n\nキャッシュカード詐欺物語を公開しました。\nキャッシュカード詐...,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",[1649201320010854401],2023-04-21T00:00:10.000Z,1649201320010854401,1649201320010854401,"{'hashtags': [{'start': 75, 'end': 80, 'tag': ...",ja,826432075825688576,NaN,NaN,NaN,NaN
95730,"[{'domain': {'id': '30', 'name': 'Entities [En...",everyone,RT @AliyaCrypto: $100 ~ 3 Days ⏳️\n\n($50)\n- ...,"{'retweet_count': 1206, 'reply_count': 0, 'lik...",[1649201295037980681],2023-04-21T00:00:04.000Z,1649201295037980681,1649201295037980681,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",en,1614227425877192705,"[{'type': 'retweeted', 'id': '1647880999601074...",NaN,NaN,NaN


In [ ]:
tweets_df.to_pickle("tweets10t.pkl")